In [95]:
import getpass
import pyodbc
import pandas as pd
from termcolor import colored, cprint
from datetime import datetime

In [96]:
def get_sql_connection(db_name):
    
    try:
        # connect to SQL Server.
        SERVER = '34.206.73.189' 
        DATABASE = db_name
        USERNAME = 'nrad' 
        PASSWORD = '83F25619-D272-4660-98A2-93AF5CC18D59' #getpass.getpass(prompt='Enter your password')
        CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
            
    except Exception as err:
            print("The Connection to SQL wasn't succesfull", err)
    
    return CNXN
            

In [97]:
def get_datetime():
    
    today = datetime.now()
    
    todaydate = today.strftime("%Y-%m-%d")
    todaydate_id = today.strftime("%Y%m%d")
    
    return todaydate, todaydate_id

In [98]:
def get_teams(conn):
    
    cursor = conn.cursor()
    
    sql =  f"""
        SELECT DISTINCT
            clientcode,
            lkupclientid,
            stlrDBName,
            leagueName
        FROM 
            ds.productyear_all
        """
    
    df_teams = pd.read_sql(sql, conn)
    
    return df_teams

In [99]:
def get_ds_scores(conn,lkupclientid_value,clientcode_value,todaydate):
    try:

        cursor = conn.cursor()
        numRecords =  f'''select count(*) as numRecords, max(cast(insertDate as date)) as maxInserDate  from ds.customerScores where lkupclientid ={lkupclientid_value} ;'''
        cntTodayScores = pd.read_sql(numRecords, conn)
        cntrecords= cntTodayScores['numRecords']
        insertdate = cntTodayScores['maxInserDate']
        
        for idx, m in enumerate(cntrecords):
            updatedScores = pd.DataFrame(columns=['clientCode_Updated', 'lkupClientId', 'NumRecords','InsertDate'])
            stalesScores = pd.DataFrame(columns=['clientCode_NotUpdated', 'lkupClientId', 'NumRecords','InsertDate'])
            value=cntrecords[idx]
            insertdate_idx = insertdate[idx]
            if str(insertdate_idx) == str(todaydate):
                updatedScores = updatedScores.append({'clientCode_updated': clientcode_value, 'lkupClientId': lkupclientid_value, 'NumRecords': value, 'InsertDate': insertdate_idx}, ignore_index=True)
                #print(str(clientcode_value)+ "-"+str(lkupclientid_value) + ":"+ " is updated in customerScores table and " + str(value) + " values has been inserted. "  )
            elif str(insertdate_idx) != str(todaydate):
                stalesScores = stalesScores.append({'clientCode_NotUpdated': clientcode_value, 'lkupClientId': lkupclientid_value, 'NumRecords': 0, 'InsertDate': insertdate_idx}, ignore_index=True)
                #print(colored(str(clientcode_value)+ "-" +str(lkupclientid_value) + ":"+ " is not updated in customerScores table. the last updateDate is: " + str(insertdate_idx), 'red', attrs=['reverse', 'blink']))
                
        conn.commit()
        cursor.close()
        
        
    except Exception as err:
            print("The Connection wasn't succesfull", err)
    print(updatedScores.head())
            
    return updatedScores,stalesScores




In [ ]:
def get_ds_scores(conn,lkupclientid_value,clientcode_value,todaydate):
    try:

        cursor = conn.cursor()
        numRecords =  f'''select count(*) as numRecords, max(cast(insertDate as date)) as maxInserDate  from ds.customerScores where lkupclientid ={lkupclientid_value} ;'''
        cntTodayScores = pd.read_sql(numRecords, conn)
        cntrecords= cntTodayScores['numRecords']
        insertdate = cntTodayScores['maxInserDate']
        
        for idx, m in enumerate(cntrecords):
            value=cntrecords[idx]
            insertdate_idx = insertdate[idx]
            if str(insertdate_idx) == str(todaydate):
                print(str(clientcode_value)+ "-"+str(lkupclientid_value) + ":"+ " is updated in customerScores table and " + str(value) + " values has been inserted. "  )
            elif str(insertdate_idx) != str(todaydate):
                print(colored(str(clientcode_value)+ "-" +str(lkupclientid_value) + ":"+ " is not updated in customerScores table. the last updateDate is: " + str(insertdate_idx), 'red', attrs=['reverse', 'blink']))
                
        conn.commit()
        cursor.close()
        
    except Exception as err:
            print("The Connection wasn't succesfull", err)




In [ ]:
def get_stlr_scores(lkupclientid_value,clientcode_value,stlrDBName_value,todaydate_id):
    
    
    try:
        SERVER = '34.206.73.189'
        DATABASE = stlrDBName_value 
        USERNAME = 'nrad' 
        PASSWORD = '83F25619-D272-4660-98A2-93AF5CC18D59' #getpass.getpass(prompt='Enter your password')
        CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
        cursor = CNXN.cursor()
        numRecords_fact =  f'''select count(*) as factnumRecords,max(dimdateid) as maxInserDate from dw.factCustomerretentionScore where lkupclientid ={lkupclientid_value} ;'''
        cntTodayScores_fact = pd.read_sql(numRecords_fact, CNXN)
        cntrecords_fact = cntTodayScores_fact['factnumRecords']
        insertdate_fact = cntTodayScores_fact['maxInserDate'] 
        for idx, m in enumerate(cntrecords_fact):
            value=cntrecords_fact[idx]
            insertdatefact_idx = insertdate_fact[idx]
            if str(insertdatefact_idx) == str(todaydate_id):
                print(str(clientcode_value)+ "-"+str(lkupclientid_value) + ":"+ " is updated and " + str(value) + " values has been inserted in factCustomerretentionScore table. "  )
            elif str(insertdatefact_idx) != str(todaydate_id):
                print(colored(str(clientcode_value)+ "-" +str(lkupclientid_value) + ":"+ " is not updated in factCustomerretentionScore table. the last updateDate is: " + str(insertdatefact_idx), 'red', attrs=['reverse', 'blink']))
                
        CNXN.commit()
        cursor.close()
    
    except Exception as err:
        print("The Connection to wasn't succesfull to " + stlrDBName_value, err)

In [93]:
def check_ds_db(df_teams):
    
    conn = get_sql_connection("DataScience")
    cursor = conn.cursor()
    
    df_teams_list = []
    for idx, team in df_teams.iterrows():
        
        sql =  f"""
            SELECT TOP (1) lkupclientid, insertDate, count(*) as count
            FROM 
              [DataScience].[DS].[customerScores]
            WHERE
              lkupclientid = {team['lkupclientid']}
            GROUP BY
              lkupclientid, insertDate
            ORDER BY
              convert(datetime, insertDate) DESC
        """
    
        df_team = pd.read_sql(sql, conn)
                 
        if (df_team.iloc[0]['insertDate'] > datetime.now().strftime("%m-%d-%Y 00:00:00")):
            df_team['current'] = True
        else:
            df_team['current'] = False
        
        df_teams_list.append(df_team)
    
    df_teams = pd.concat(df_teams_list)
    df_teams = df_teams.sort_values(by=['lkupclientid'])
    
    return df_teams

In [105]:
if __name__ == "__main__":
    
    # get the current date and date_id
    todaydate, todaydate_id = get_datetime()
        
    # get a connection to sql
    conn = get_sql_connection("DataScience")
    
    # get a list of teams from DataScience table
    teams = get_teams(conn)
    
    # add the insert date from the data science table
    for team in teams.iterrows():
        
        
    
    

<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


### DataScience Database
66ers-19: is not updated in customerScores table. the last updateDate is: 2022-03-14
admirals-16: is updated in customerScores table and 122734 values has been inserted. 

### Stlr Databases
66ers-19: is not updated in customerScores table. the last updateDate is: 2022-03-14
admirals-16: is updated in customerScores table and 122734 values has been inserted. 

### Parameters
- clientcode
- lkupclientid
- count
- insertDate